In [1]:
using Pkg, Revise
Pkg.activate("../.")
using BasicBSpline
using StaticArrays
using Plots
using LinearAlgebra
using Meshes

  Activating project at `~/MEGA/EMAP/Julia_Tutorials`


## Meshes

A mesh $M = (V, E, F)$ is a list of vertices $V$, non-oriented edges $E$ and oriented faces $F$.
One usually uses *half-edge* to refer to an oriented edge, and *edge* for an oriented one.

In [112]:
struct Vertex
    v::Vec{3, Real}
end

Vertex(x,y,z) = Vertex([x,y,z])
Vertex(x,y) = Vertex([x,y,0])
Vertex(v::Vector{2}) = Vertex(vcat(v,[0]))

struct Halfedge
    h::Vec{2,Vertex}
end
Halfedge(v1,v2) = Halfedge([v1,v2])
struct Edge
    e::Vec{2,Halfedge}
end
Edge(h1::Halfedge) = Edge([h1,Halfedge(h1.h[2],h1.h[1])])
Edge(h1::Halfedge,h2::Halfedge) = Edge([h1,h2])
Edge(v1::Vertex, v2::Vertex) = Edge(Halfedge(v1,v2), Halfedge(v2,v1))

struct Face
    f::Vec{3,Vertex}
end

struct Mesh
    V::Vector{Vertex}
    E::Vector{Edge}
    F::Vector{Face}
end

function Mesh(V::Vector{Vertex}, faces::Vector{<:Tuple}) 
    F = []
    E = []
    H = Dict()
    for f in faces
        v1,v2,v3 = V[f[1]],V[f[2]],V[f[3]]
        e1 = Edge(v1,v2)
        e2 = Edge(v2,v3)
        e3 = Edge(v3,v1)
        E = vcat([e1,e2,e3],E)
        F = vcat([Face([v1,v2,v3])],F)
    end
   Mesh(V,E,F) 
end
Mesh(V::Vector{Vertex}, faces::Tuple)  = Mesh(V, [faces])


Base.getindex(x::Vertex, i) = getproperty(x, fieldnames(typeof(x))[1])[i]
Base.getindex(x::Halfedge, i) = getproperty(x, fieldnames(typeof(x))[1])[i]
Base.getindex(x::Halfedge, i, j) = getproperty(x, fieldnames(typeof(x))[1])[i][j]
Base.getindex(x::Edge, i) = getproperty(x, fieldnames(typeof(x))[1])[i]
Base.getindex(x::Edge, i, j) = getproperty(x, fieldnames(typeof(x))[1])[i][j]
Base.getindex(x::Edge, i, j, k) = getproperty(x, fieldnames(typeof(x))[1])[i][j][k]
Base.getindex(x::Face, i) = getproperty(x, fieldnames(typeof(x))[1])[i]

Base.:-(v1::Vertex, v2::Vertex) = v1.v - v2.v
Base.:+(v1::Vertex, v2::Vertex) = v1.v + v2.v
Base.:+(v1::Vertex, v2::Vertex...) = v1.v + +(v2...)
Base.:-(v1::Vertex, v2::Vertex...) = v1.v - -(v2...)
Base.length(v::Vertex) = length(v.v)
M = Mesh([v1,v2,v3,v4],[(1,2,3),(1,4,3)])

Mesh(Vertex[Vertex(Real[0, 0, 0]), Vertex(Real[1, 0, 0]), Vertex(Real[0, 1, 0]), Vertex(Real[-1, 0, 0])], Edge[Edge(Halfedge[Halfedge(Vertex[Vertex(Real[0, 0, 0]), Vertex(Real[-1, 0, 0])]), Halfedge(Vertex[Vertex(Real[-1, 0, 0]), Vertex(Real[0, 0, 0])])]), Edge(Halfedge[Halfedge(Vertex[Vertex(Real[-1, 0, 0]), Vertex(Real[0, 1, 0])]), Halfedge(Vertex[Vertex(Real[0, 1, 0]), Vertex(Real[-1, 0, 0])])]), Edge(Halfedge[Halfedge(Vertex[Vertex(Real[0, 1, 0]), Vertex(Real[0, 0, 0])]), Halfedge(Vertex[Vertex(Real[0, 0, 0]), Vertex(Real[0, 1, 0])])]), Edge(Halfedge[Halfedge(Vertex[Vertex(Real[0, 0, 0]), Vertex(Real[1, 0, 0])]), Halfedge(Vertex[Vertex(Real[1, 0, 0]), Vertex(Real[0, 0, 0])])]), Edge(Halfedge[Halfedge(Vertex[Vertex(Real[1, 0, 0]), Vertex(Real[0, 1, 0])]), Halfedge(Vertex[Vertex(Real[0, 1, 0]), Vertex(Real[1, 0, 0])])]), Edge(Halfedge[Halfedge(Vertex[Vertex(Real[0, 1, 0]), Vertex(Real[0, 0, 0])]), Halfedge(Vertex[Vertex(Real[0, 0, 0]), Vertex(Real[0, 1, 0])])])], Face[Face(Vertex[Ver

In [102]:
M.E

3-element Vector{Edge}:
 Edge(Halfedge[Halfedge(Vertex[Vertex(Real[0, 0, 0]), Vertex(Real[1, 0, 0])]), Halfedge(Vertex[Vertex(Real[1, 0, 0]), Vertex(Real[0, 0, 0])])])
 Edge(Halfedge[Halfedge(Vertex[Vertex(Real[1, 0, 0]), Vertex(Real[0, 1, 0])]), Halfedge(Vertex[Vertex(Real[0, 1, 0]), Vertex(Real[1, 0, 0])])])
 Edge(Halfedge[Halfedge(Vertex[Vertex(Real[0, 1, 0]), Vertex(Real[0, 0, 0])]), Halfedge(Vertex[Vertex(Real[0, 0, 0]), Vertex(Real[0, 1, 0])])])

In [106]:
v1 = Vertex([0,0,0])
v2 = Vertex(1,0,0)
v3 = Vertex(0,1,0)
v4 = Vertex(-1,0,0)

h1 = Halfedge(v1,v2)
h2 = Halfedge(v2,v3)
h3 = Halfedge(v3,v1)

e1 = Edge(h1)
e2 = Edge(h2)
e3 = Edge(h3)

f1 = Face([v1,v2,v3])
f2 = Face([v1,v4,v3])

M = Mesh([v1,v2,v3],[e1,e2,e3],[f1])
M = Mesh([v1,v2,v3,v4],[(1,2,3),(1,4,3)])

Mesh(Vertex[Vertex(Real[0, 0, 0]), Vertex(Real[1, 0, 0]), Vertex(Real[0, 1, 0]), Vertex(Real[-1, 0, 0])], Edge[Edge(Halfedge[Halfedge(Vertex[Vertex(Real[0, 0, 0]), Vertex(Real[-1, 0, 0])]), Halfedge(Vertex[Vertex(Real[-1, 0, 0]), Vertex(Real[0, 0, 0])])]), Edge(Halfedge[Halfedge(Vertex[Vertex(Real[-1, 0, 0]), Vertex(Real[0, 1, 0])]), Halfedge(Vertex[Vertex(Real[0, 1, 0]), Vertex(Real[-1, 0, 0])])]), Edge(Halfedge[Halfedge(Vertex[Vertex(Real[0, 1, 0]), Vertex(Real[0, 0, 0])]), Halfedge(Vertex[Vertex(Real[0, 0, 0]), Vertex(Real[0, 1, 0])])]), Edge(Halfedge[Halfedge(Vertex[Vertex(Real[0, 0, 0]), Vertex(Real[1, 0, 0])]), Halfedge(Vertex[Vertex(Real[1, 0, 0]), Vertex(Real[0, 0, 0])])]), Edge(Halfedge[Halfedge(Vertex[Vertex(Real[1, 0, 0]), Vertex(Real[0, 1, 0])]), Halfedge(Vertex[Vertex(Real[0, 1, 0]), Vertex(Real[1, 0, 0])])]), Edge(Halfedge[Halfedge(Vertex[Vertex(Real[0, 1, 0]), Vertex(Real[0, 0, 0])]), Halfedge(Vertex[Vertex(Real[0, 0, 0]), Vertex(Real[0, 1, 0])])])], Face[Face(Vertex[Ver

In [110]:
M.E

6-element Vector{Edge}:
 Edge(Halfedge[Halfedge(Vertex[Vertex(Real[0, 0, 0]), Vertex(Real[-1, 0, 0])]), Halfedge(Vertex[Vertex(Real[-1, 0, 0]), Vertex(Real[0, 0, 0])])])
 Edge(Halfedge[Halfedge(Vertex[Vertex(Real[-1, 0, 0]), Vertex(Real[0, 1, 0])]), Halfedge(Vertex[Vertex(Real[0, 1, 0]), Vertex(Real[-1, 0, 0])])])
 Edge(Halfedge[Halfedge(Vertex[Vertex(Real[0, 1, 0]), Vertex(Real[0, 0, 0])]), Halfedge(Vertex[Vertex(Real[0, 0, 0]), Vertex(Real[0, 1, 0])])])
 Edge(Halfedge[Halfedge(Vertex[Vertex(Real[0, 0, 0]), Vertex(Real[1, 0, 0])]), Halfedge(Vertex[Vertex(Real[1, 0, 0]), Vertex(Real[0, 0, 0])])])
 Edge(Halfedge[Halfedge(Vertex[Vertex(Real[1, 0, 0]), Vertex(Real[0, 1, 0])]), Halfedge(Vertex[Vertex(Real[0, 1, 0]), Vertex(Real[1, 0, 0])])])
 Edge(Halfedge[Halfedge(Vertex[Vertex(Real[0, 1, 0]), Vertex(Real[0, 0, 0])]), Halfedge(Vertex[Vertex(Real[0, 0, 0]), Vertex(Real[0, 1, 0])])])

In [47]:
∂(h::Halfedge) = h[2] - h[1]
∂(f::Face) = [Halfedge(f[1],f[2]),Halfedge(f[2],f[3]),Halfedge(f[3],f[1])]
n(f::Face) = (f[2] - f[1]) × (f[3] - f[1])/norm((f[2] - f[1]) × (f[3] - f[1]))

dual(h::Halfedge) = Halfedge(h[2],h[1])

dual (generic function with 1 method)

In [73]:
@show ∂(h1)
@show dual(h1)
@show n(f1)
@show ∂(f1);

∂(h1) = [-1, -2, 0]
dual(h1) = Halfedge(Vertex[Vertex(Real[0, 0, 0]), Vertex(Real[1, 2, 0])])
n(f1) = [0.0, 0.0, 1.0]
∂(f1) = Halfedge[Halfedge(Vertex[Vertex(Real[1, 2, 0]), Vertex(Real[0, 0, 0])]), Halfedge(Vertex[Vertex(Real[0, 0, 0]), Vertex(Real[1, 1, 0])]), Halfedge(Vertex[Vertex(Real[1, 1, 0]), Vertex(Real[1, 2, 0])])]
